<a href="https://colab.research.google.com/github/sayedmahamudsijan/digit-counter/blob/main/Count_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python-headless tqdm


In [2]:
import os
import zipfile
import urllib.request

# Download ZIP file
url = "https://www.dropbox.com/s/y9bs6r4nc7pj48c/digits.zip?dl=1"
zip_path = "digits.zip"

print("Downloading ZIP file...")
urllib.request.urlretrieve(url, zip_path)

# Extract
extract_dir = "digits"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted to:", extract_dir)


Extracted to: digits


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import os
import cv2
import numpy as np
from tqdm import tqdm

# --- Load and preprocess MNIST ---
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train_cat = to_categorical(y_train, 10)

# --- Build model ---
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Train model ---
print("Training model on MNIST...")
model.fit(x_train, y_train_cat, epochs=3, batch_size=64)

# --- Predict digits from folder ---
digit_counts = [0] * 10
folder = "/content/digits/digits"  # update path if needed

print("\nClassifying digits...")
for filename in tqdm(sorted(os.listdir(folder))):
    path = os.path.join(folder, filename)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    img_resized = cv2.resize(img, (28, 28))
    img_norm = img_resized.astype('float32') / 255.0
    img_input = img_norm.reshape(1, 28, 28, 1)

    pred = np.argmax(model.predict(img_input, verbose=0))
    digit_counts[pred] += 1

# --- Output results ---
print("\n📊 Digit counts:", digit_counts)
print("🔢 Total images:", sum(digit_counts))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model on MNIST...
Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 53s 55ms/step - accuracy: 0.8641 - loss: 0.4412
Epoch 2/3
357/938 ━━━━━━━━━━━━━━━━━━━━ 33s 58ms/step - accuracy: 0.9831 - loss: 0.0579


Classifying digits...


100%|██████████| 12001/12001 [21:14<00:00,  9.42it/s]


📊 Digit counts: [1254, 1451, 941, 1511, 1108, 1240, 1221, 942, 951, 1381]
🔢 Total images: 12000
📈 Model accuracy on MNIST: 0.9848
